In [27]:
import rclpy
from rclpy.node import Node
from turtlesim.srv import Spawn
from geometry_msgs.msg import Twist
import random
import time

rclpy.init()
node = Node('turtle_controller')

# create spawn client
spawn_client = node.create_client(Spawn, 'spawn')
while not spawn_client.wait_for_service(timeout_sec=1.0):
    node.get_logger().info('Waiting for spawn service...')

publisher = None

In [30]:
# random coordinates and orientation
x = random.uniform(2.0, 9.0)  # avoid edges
y = random.uniform(2.0, 9.0)
theta = random.uniform(0.0, 6.28)  # 0 to 2*pi

# spawn request
request = Spawn.Request()
request.x = x
request.y = y
request.theta = theta
request.name = f'turtle_{random.randint(1, 1000)}'

# call spawn
future = spawn_client.call_async(request)
rclpy.spin_until_future_complete(node, future)

if future.result() is not None:
    turtle_name = future.result().name
    node.get_logger().info(f'Spawned new turtle: {turtle_name} at ({x:.2f}, {y:.2f})')
    publisher = node.create_publisher(Twist, f'/{turtle_name}/cmd_vel', 10) # publisher for the new turtle
else:
    node.get_logger().error('Failed to spawn turtle.')


[INFO] [1734692336.478835545] [turtle_controller]: Spawned new turtle: turtle_585 at (5.52, 3.47)


In [31]:
if publisher is None:
    node.get_logger().error('No publisher available. Spawn a turtle first.')
    publisher = node.create_publisher(Twist, f'/turtle1/cmd_vel', 10)

radius = 1.0
angular_velocity = 1.0  # rad/s
linear_velocity = radius * angular_velocity  # v = r * ω

twist = Twist()
twist.linear.x = linear_velocity  # forward
twist.angular.z = angular_velocity  # angular

# circle duration = 2 * pi / angular
duration = 2 * 3.14159 / angular_velocity
node.get_logger().info(f'Moving in a circle for {duration:.2f} seconds.')

start_time = node.get_clock().now()
while (node.get_clock().now() - start_time).nanoseconds < duration * 1e9:
    publisher.publish(twist)
    time.sleep(0.1)

# stop turtle
twist.linear.x = 0.0
twist.angular.z = 0.0
publisher.publish(twist)
node.get_logger().info('Completed circular trajectory.')


[INFO] [1734692339.316711180] [turtle_controller]: Moving in a circle for 6.28 seconds.
[INFO] [1734692345.658921405] [turtle_controller]: Completed circular trajectory.


True

In [26]:
if node is not None:
    node.destroy_node()
    rclpy.shutdown()
    print("ROS 2 node successfully shut down.")
else:
    print("No active ROS 2 node to shut down.")

ROS 2 node successfully shut down.
